# Training the VAE

## Setup

In [1]:
%load_ext tensorboard

In [2]:
import os

os.environ["KERAS_BACKEND"] = "tensorflow"
# import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import keras
from keras import layers, ops
from data import vae_data  # hurray for modularity!

2024-12-08 20:47:52.306904: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-08 20:48:04.449201: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-12-08 20:48:04.518552: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus

In [3]:
import datetime
curr_datetime = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
log_dir = "logs/vae/" + curr_datetime
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

## Create a custom sampling layer and VAE model

In [4]:
class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding our compressed data."""

    def __init__(self, kl_loss_factor=1, seed=None, **kwargs):
        super().__init__(**kwargs)
        self.seed_generator = keras.random.SeedGenerator(seed=seed)
        self.kl_loss_factor = kl_loss_factor

    def call(self, inputs):
        z_mean, z_log_var = inputs
        # add loss
        kl_loss = -0.5 * (1 + z_log_var - ops.square(z_mean) - ops.exp(z_log_var))
        self.add_loss(kl_loss * self.kl_loss_factor)
        # sample
        batch = ops.shape(z_mean)[0]
        dim = ops.shape(z_mean)[1]
        epsilon = keras.random.normal(shape=(batch, dim), seed=self.seed_generator)
        return z_mean + ops.exp(0.5 * z_log_var) * epsilon


class VAE(keras.Model):
    """Wraps an encoder and decoder into a single variational autoencoder model."""

    def __init__(self, encoder, decoder, **kwargs):
        # TODO hacky?
        vae_inputs = keras.Input(shape=encoder.input.shape[1:], name="vae_inputs")
        vae_outputs = decoder(encoder(vae_inputs))
        super().__init__(inputs=vae_inputs, outputs=vae_outputs, **kwargs)
        self.encoder = encoder
        self.decoder = decoder


# need to serialize because our 2nd LSTM will use the encoder
@keras.saving.register_keras_serializable()
class Log1p(layers.Layer):
    def call(self, inputs):
        return ops.log1p(inputs)


class Expm1(layers.Layer):
    def call(self, inputs):
        return ops.expm1(inputs)

## Build the models

In [5]:
input_shape = (6,)
latent_dim = 2

In [6]:
encoder_inputs = keras.Input(shape=input_shape, name="encoder_inputs")

x = Log1p(name="log_transform")(encoder_inputs)

x = layers.Dense(5, activation="relu")(x)
encoder_outputs_mean = layers.Dense(latent_dim, name="encoder_outputs_mean")(x)
encoder_outputs_log_var = layers.Dense(latent_dim, name="encoder_outputs_log_var")(x)

encoder = keras.Model(
    inputs=encoder_inputs,
    outputs=[encoder_outputs_mean, encoder_outputs_log_var],
    name="encoder",
)
encoder.summary()

2024-12-08 20:48:05.249021: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


Model: "encoder"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ encoder_inputs      │ (None, 6)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ log_transform       │ (None, 6)         │          0 │ encoder_inputs[0… │
│ (Log1p)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 5)         │         35 │ log_transform[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ encoder_outputs_me… │ (None, 2)         │         12 │ dense[0][0]       │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ encoder_outputs_lo… │ (None, 2)         │         12 │ dense[0][0]       │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 59 (236.00 B)

 Trainable params: 59 (236.00 B)

 Non-trainable params: 0 (0.00 B)

In [7]:
decoder_inputs_mean = keras.Input(shape=(latent_dim,), name="decoder_inputs_mean")
decoder_inputs_log_var = keras.Input(shape=(latent_dim,), name="decoder_inputs_log_var")

latent_sample = Sampling(kl_loss_factor=1, seed=489, name="latent_sample")([
    decoder_inputs_mean,
    decoder_inputs_log_var,
])

x = layers.Dense(5, activation="relu")(latent_sample)
x = Expm1(name="exp_transform")(x)
decoder_outputs = layers.Dense(6, name="decoder_outputs")(x)

decoder = keras.Model(
    inputs=[decoder_inputs_mean, decoder_inputs_log_var],
    outputs=decoder_outputs,
    name="decoder",
)
decoder.summary()

Model: "decoder"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ decoder_inputs_mean │ (None, 2)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_inputs_log… │ (None, 2)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ latent_sample       │ (None, 2)         │          0 │ decoder_inputs_m… │
│ (Sampling)          │                   │            │ decoder_inputs_l… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 5)         │         15 │ latent_sample[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ exp_transform       │ (None, 5)         │          0 │ dense_1[0][0]     │
│ (Expm1)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_outputs     │ (None, 6)         │         36 │ exp_transform[0]… │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 51 (204.00 B)

 Trainable params: 51 (204.00 B)

 Non-trainable params: 0 (0.00 B)

In [8]:
vae = VAE(encoder=encoder, decoder=decoder, name="vae")
vae.summary()

Model: "vae"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ vae_inputs          │ (None, 6)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ encoder             │ [(None, 2),       │         59 │ vae_inputs[0][0]  │
│ (Functional)        │ (None, 2)]        │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder             │ (None, 6)         │         51 │ encoder[0][0],    │
│ (Functional)        │                   │            │ encoder[0][1]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 110 (440.00 B)

 Trainable params: 110 (440.00 B)

 Non-trainable params: 0 (0.00 B)

## Train the VAE

In [9]:
train, valid, test = vae_data

In [10]:
vae.compile(
    optimizer=keras.optimizers.AdamW(learning_rate=1e-4, clipnorm=1e-6),
    loss="mean_squared_error",
)

history = vae.fit(
    train,
    epochs=100,
    validation_data=valid,
    callbacks=[
        tensorboard_callback,
        keras.callbacks.EarlyStopping(patience=5, start_from_epoch=10),
        keras.callbacks.ModelCheckpoint(filepath="models/checkpoints/" + curr_datetime + "-{epoch:03d}.keras"),
    ],
)

Epoch 1/100


2024-12-08 20:48:07.300543: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x12a64380 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-12-08 20:48:07.300580: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3060 Laptop GPU, Compute Capability 8.6
2024-12-08 20:48:07.339575: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-12-08 20:48:07.540904: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600


     93/Unknown 3s 2ms/step - loss: 283496218624.0000

2024-12-08 20:48:08.615936: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


   3725/Unknown 18s 4ms/step - loss: 333981548544.0000

...

3725/3725 ━━━━━━━━━━━━━━━━━━━━ 13s 3ms/step - loss: 17318925519076458496.0000 - val_loss: 6943875.5000
Epoch 97/100
   1/3725 ━━━━━━━━━━━━━━━━━━━━ 6:16 101ms/step - loss: 1431970.6250

2024-12-08 21:14:46.616901: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous recv item cancelled. Key hash: 6666645939987620405
2024-12-08 21:14:46.616974: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous recv item cancelled. Key hash: 6560117268687586076


3701/3725 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 2337168920122753024.0000

2024-12-08 21:14:58.581474: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous recv item cancelled. Key hash: 6666645939987620405
2024-12-08 21:14:58.581527: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous recv item cancelled. Key hash: 6560117268687586076
2024-12-08 21:15:00.623608: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous recv item cancelled. Key hash: 6666645939987620405
2024-12-08 21:15:00.623677: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous recv item cancelled. Key hash: 6560117268687586076


3725/3725 ━━━━━━━━━━━━━━━━━━━━ 14s 4ms/step - loss: 2366293608753004544.0000 - val_loss: 5871981.5000


In [11]:
test_stats = vae.evaluate(test, return_dict=True)

220/220 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 5341759.5000


2024-12-08 21:15:02.841735: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous recv item cancelled. Key hash: 6666645939987620405
2024-12-08 21:15:02.841783: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous recv item cancelled. Key hash: 6560117268687586076


In [13]:
encoder.save(f"encoder.keras")  # only run if things went well!